### Aluno: Luiz Zairo Bastos Viana - 586107

1. Importação das bibliotecas

In [1]:
import duckdb
import pandas as pd
import re

2. Leitura da base de dados e convertendo para parquet

b) Removendo os trava-zap

In [2]:
dataset = pd.read_pickle("./data/fakeTelegram.BR_2022_clean.pkl")

c) Exportando os dados para o `.parquet`

In [3]:
duckdb.register("telegram_df", dataset)

# Exporta para Parquet
duckdb.execute("""
    COPY telegram_df TO './data/fakeTelegram.BR_2022.parquet' (FORMAT 'parquet');
""")


d) Exportando os dados para o `DuckDB`

In [4]:
duckdb.sql("SELECT * FROM read_parquet('./data/fakeTelegram.BR_2022.parquet')")


┌─────────────────────┬──────────────────────────────────┬──────────────────────────────────┬──────────────────────────────────────┬────────────┬─────────────────────────────────────────────────────────────────┬───────────┬───────────────┬───────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

e) Utilizando o `DuckDB`recupere:
  1. A quantidade de Mensagens

In [5]:
tamanho_dataset = duckdb.sql("SELECT COUNT(*) AS tamanho FROM telegram_df").fetchone()[0]
print(f"Quantidade de mensagens do dataset: {tamanho_dataset}")

Quantidade de mensagens do dataset: 550570


  2. Quantidade de Usuários

In [6]:
unique_count = duckdb.sql("SELECT COUNT(DISTINCT id_member_anonymous) AS unique_count FROM telegram_df").fetchone()[0]
print(f"Quantidade de usuários: {unique_count}")

Quantidade de usuários: 14696


  3. Quantidade de grupos

In [7]:
unique_count = duckdb.sql("SELECT COUNT(DISTINCT id_group_anonymous) AS unique_count FROM telegram_df").fetchone()[0]
print(f"Quantidade de grupos: {unique_count}")

Quantidade de grupos: 178


4. Quantidade de mensagens que possuem apenas texto

In [8]:
qtd_mensagens_nao_nulas = duckdb.sql("SELECT COUNT(*) AS qtd_mensagens_nao_nulas FROM telegram_df WHERE text_content_anonymous <> '' ").fetchone()[0]
print(f"Quantidade de mensagens que possuem apenas texto: {qtd_mensagens_nao_nulas}")

Quantidade de mensagens que possuem apenas texto: 437201


  5. Quantidade de mensagens contendo mídias

In [9]:
qtd_has_media_true = duckdb.sql("SELECT COUNT(*) AS qtd_has_media_true FROM telegram_df WHERE has_media = TRUE").fetchone()[0]
print(f"Quantidade de valores True na coluna 'has_media': {qtd_has_media_true}")

Quantidade de valores True na coluna 'has_media': 331159


  6. Quantidade de mensagens por tipo de mídia (jpg, mp4 etc)

In [11]:
media_type_counts = duckdb.sql("SELECT media_type, COUNT(media_type) AS count FROM telegram_df WHERE has_media = TRUE GROUP BY media_type ORDER BY count DESC").df()
print(media_type_counts)

                                 media_type   count
0                                 image/jpg  199917
1                                       url   99942
2                                 video/mp4   18494
3   application/vnd.android.package-archive    7159
4                           application/pdf    2850
..                                      ...     ...
62   image/08e8201b6f32c358808c7ac89d8c8422       1
63                        application/x-msi       1
64                                audio/m4a       1
65                           audio/x-ms-wma       1
66   image/af75130da3b6516eb3c1eb9ab62b5219       1

[67 rows x 2 columns]


Os seguintes itens não possui dados referentes no dataset fornecido:  
  7. Quantidade de mensagens por estado;  
  8. Quantidade de usuários por estado;  
  9. Relação quantidade de usuários por quantidade de mensagens por estado;  
  10.  Quantidade de mensagens por país;  
  11.  Quantidade de mensagens Brasil X Países Estrangeiros;  

  12. As 30 URLs que mais se repetem (mais compartilhadas)

In [12]:
top_30_media_urls = duckdb.sql("""
  SELECT media_url, COUNT(*) AS count
  FROM telegram_df
  WHERE media_url IS NOT NULL
  GROUP BY media_url
  ORDER BY count DESC
  LIMIT 30
""").df()

print(top_30_media_urls)

                                            media_url  count
0                                   t.me/alexeconomia   4112
1     https://youtube.com/c/especulandoosfatosoficial   1607
2                                   t.me/fimtaproximo   1581
3                              t.me/+EWlGMatRZGg3OTlh   1419
4                        https://youtu.be/qbTzhB0akt8   1160
5                        https://youtu.be/zDuOoyhyN-4   1020
6                              t.me/+ewZIPdZ42vEyNzJh    772
7                https://t.me/canalselvabrasiloficial    707
8                        https://youtu.be/4DHk9KZ01HM    660
9                           T.me/Arthur_Senna_Trader2    639
10                       https://youtu.be/x2uiakywcrI    576
11                             t.me/+bseuTTFOO8c5YWFh    515
12                          https://t.me/Ovetoroculto    394
13  https://noticias.uol.com.br/politica/ultimas-n...    387
14          https://t.me/ContagemDeVotosPRO_BOLSONARO    375
15                      

  13. Os 30 domínios que mais se repetem (mais compartilhados)

In [18]:
dominios = duckdb.sql("""
  SELECT 
    regexp_extract(media_url, '^(?:https?://)?(?:www\\.)?([^/]+)', 1) AS dominio, 
    COUNT(media_url) AS quantidade
  FROM telegram_df
  WHERE media_url IS NOT NULL
  GROUP BY dominio
  ORDER BY quantidade DESC
  LIMIT 30
""").df()

print(dominios)

                        dominio  quantidade
0                      youtu.be       52618
1                          t.me       19436
2                   youtube.com       14468
3       terrabrasilnoticias.com        8280
4                 instagram.com        6588
5   jornaldacidadeonline.com.br        4801
6                   twitter.com        4435
7           gazetabrasil.com.br        1777
8                    pleno.news        1600
9                  facebook.com        1366
10                 bitchute.com        1174
11                         T.me         992
12  aliadosbrasiloficial.com.br         945
13             revistaoeste.com         928
14               kwai-video.com         863
15                    gettr.com         840
16           vistapatria.com.br         830
17      verdadecensurada.com.br         829
18       conexaopolitica.com.br         746
19                vm.tiktok.com         744
20                m.youtube.com         693
21          noticias.uol.com.br 

  14. Os 30 usuários mais ativos

In [19]:
top_30_users = duckdb.sql("""
  SELECT id_member_anonymous, COUNT(*) AS count
  FROM telegram_df
  WHERE id_member_anonymous IS NOT NULL
  GROUP BY id_member_anonymous
  ORDER BY count DESC
  LIMIT 30
""").df()

print(top_30_users)

                 id_member_anonymous  count
0   abe534d581ec6d552243d6955d3c3cd8  12289
1   1665e22b0f564cd46d343f7677014821   5452
2   1ac091b8ed5c4e42383f1b4ff4cc9b2d   5031
3   c743967449a387ad2c1c7e03b2c45b36   2793
4   e7998863ac2a40086657fab4a6b463c9   1911
5   e8fd8fee8c39342d37993775da7756d5   1703
6   2f4be6244ede15b46e8329a2c975be30   1620
7   d49f81df0c75d1d72bee6c5b2d707da0   1565
8   4f7d493f0f6222d56e5b19a4f7c336cc   1435
9   773b9bd5b02a2e96f9d732c29bfcb663   1407
10  c052c859b42c5a1923c22f5a201de746   1374
11  f233cf8b1d4ede06f32199a7e0081bf5   1257
12  56b8359fd127312651b80b8ed8030085   1198
13  e5e3e38d433016adf62ea34432853a3c   1088
14  fa582c9e95da255e44f4c198d0daf3df   1029
15  cf061f911f3df59fca0b3f084abe88cc    963
16  3e49fd40fd973ee1b8f1a6d58feb4a54    955
17  bd7859770eef94f34d0344a5397c5443    942
18  f4561a21fb26cfbebf48dcd5c557f82f    939
19  40743c2ea5139af088057763ab069b1a    933
20  beb77036ff26e78865da19310ac95169    890
21  50cc2b20a3c2c24ce68bb2c0554d

  15. Os 30 usuários que mais compartilharam texto

In [26]:
top_30_text_users = duckdb.sql("""
  SELECT id_member_anonymous, COUNT(*) AS count
  FROM telegram_df
  WHERE id_member_anonymous IS NOT NULL AND text_content_anonymous <> '' AND text_content_anonymous IS NOT NULL
  GROUP BY id_member_anonymous
  ORDER BY count DESC
  LIMIT 30
""").df()

print(top_30_text_users)

                 id_member_anonymous  count
0   abe534d581ec6d552243d6955d3c3cd8  12289
1   1665e22b0f564cd46d343f7677014821   5452
2   1ac091b8ed5c4e42383f1b4ff4cc9b2d   3993
3   c743967449a387ad2c1c7e03b2c45b36   2793
4   d49f81df0c75d1d72bee6c5b2d707da0   1565
5   e7998863ac2a40086657fab4a6b463c9   1384
6   4f7d493f0f6222d56e5b19a4f7c336cc   1353
7   773b9bd5b02a2e96f9d732c29bfcb663   1309
8   e8fd8fee8c39342d37993775da7756d5   1222
9   f233cf8b1d4ede06f32199a7e0081bf5   1133
10  c052c859b42c5a1923c22f5a201de746   1065
11  e5e3e38d433016adf62ea34432853a3c    903
12  cf061f911f3df59fca0b3f084abe88cc    849
13  fa582c9e95da255e44f4c198d0daf3df    838
14  56b8359fd127312651b80b8ed8030085    819
15  beb77036ff26e78865da19310ac95169    711
16  a378e9743fa3ca297df321cfa0e7cf6a    699
17  566ba5c5eeeac2fc18b65965c3909a1f    673
18  40743c2ea5139af088057763ab069b1a    647
19  7696d5103cdb8ac352d748a1db1126b0    640
20  79fa510c3ab44548d903284ab9162c9d    639
21  fbbbfb737acb18812f7de2400841

  16. Os 30 usuários que mais compartilharam mídias

In [27]:
top_30_media_users = duckdb.sql("""
  SELECT id_member_anonymous, COUNT(*) AS count
  FROM telegram_df
  WHERE has_media = TRUE AND has_media_url = TRUE AND id_member_anonymous IS NOT NULL
  GROUP BY id_member_anonymous
  ORDER BY count DESC
  LIMIT 30
""").df()

print(top_30_media_users)

                 id_member_anonymous  count
0   1665e22b0f564cd46d343f7677014821   2956
1   1ac091b8ed5c4e42383f1b4ff4cc9b2d   2105
2   4f7d493f0f6222d56e5b19a4f7c336cc   1053
3   773b9bd5b02a2e96f9d732c29bfcb663    982
4   c052c859b42c5a1923c22f5a201de746    761
5   cf061f911f3df59fca0b3f084abe88cc    607
6   e7998863ac2a40086657fab4a6b463c9    530
7   53e74af95497f943c524abaaa9482e33    468
8   beb77036ff26e78865da19310ac95169    458
9   9794a13f91433216821ad18624372498    458
10  5d321b03e74b751ac577ef022350306d    453
11  762b3696b26db306fbd2022412c47bd1    437
12  56b8359fd127312651b80b8ed8030085    419
13  a7e85072244cae15446c9d517dc01a1a    378
14  79fa510c3ab44548d903284ab9162c9d    359
15  40743c2ea5139af088057763ab069b1a    354
16  ffbb8aa82b50cd4fab497d5ce36627d7    343
17  566ba5c5eeeac2fc18b65965c3909a1f    335
18  6134f475127170f39fe48d53f874080e    335
19  f08cb953e68eddae5d1e15edf8c8df0b    327
20  eaef2dbdc2914b034116569bc8919403    326
21  e5e3e38d433016adf62ea3443285

  17. As 30 mensagens mais compartilhadas  

In [34]:
combined_top_30 = duckdb.sql("""
  SELECT 'media' AS column_name, media AS value, COUNT(*) AS count
  FROM telegram_df
  WHERE media IS NOT NULL
  GROUP BY media
  UNION ALL
  SELECT 'text_content_anonymous' AS column_name, text_content_anonymous AS value, COUNT(*) AS count
  FROM telegram_df
  WHERE text_content_anonymous IS NOT NULL AND text_content_anonymous <> ''
  GROUP BY text_content_anonymous
  ORDER BY count DESC
  LIMIT 30
""").df()

display(combined_top_30)


,column_name,value,count
0,text_content_anonymous,This community was blocked in Brazil following...,17422
1,media,b5ab97b82ce9a3957022b7388d8fb910.apk,5473
2,media,4d60b50f292b14a9d62a0bb2f1b42796.jpg,1846
3,media,5d4089a914ac6fa7d04cbe08bb8057f5.apk,1481
4,media,c4266d6971f2e3517f99d5736d8f0f40.jpg,1404
5,media,baba583ff1a3a79f11ed019cc319f2f2.jpg,1136
6,text_content_anonymous,Rough_sex🙈,1134
7,text_content_anonymous,Anal sex🙈,1118
8,text_content_anonymous,سکس مردان ازبک با زن انگلیسی با این vpn از سای...,1019
9,media,7786c121f410f21595063c22689e2866.jpg,764


  18. As 30 mensagens mais compartilhadas em grupos diferentes

In [40]:
top_30_messages_groups = duckdb.sql("""
  SELECT id_message, COUNT(DISTINCT id_group_anonymous) AS group_count
  FROM telegram_df
  GROUP BY id_message
  ORDER BY group_count DESC
  LIMIT 30
""").df()

print(top_30_messages_groups)


    id_message  group_count
0         1396           18
1         1023           18
2         1638           18
3         1614           18
4         1031           18
5         1012           18
6         1395           18
7         1399           17
8         1285           17
9         1013           17
10        1099           17
11        1116           17
12        1032           17
13        1382           17
14        1006           17
15        1434           17
16        1014           17
17         804           17
18        2694           16
19         938           16
20         674           16
21        1015           16
22        1438           16
23        1376           16
24        1100           16
25        1606           16
26        1076           16
27         635           16
28        2237           16
29         782           16


  19. Mensagens  idênticas  compartilhadas  pelo  mesmo  usuário  (e  suas quantidades)

In [42]:
identical_messages = duckdb.sql("""
  SELECT 
    id_member_anonymous, 
    text_content_anonymous, 
    COUNT(*) AS quantidade
  FROM telegram_df
  WHERE text_content_anonymous IS NOT NULL AND text_content_anonymous <> '' AND id_member_anonymous IS NOT NULL
  GROUP BY id_member_anonymous, text_content_anonymous
  HAVING COUNT(*) > 1
  ORDER BY quantidade DESC
""").df()

display(identical_messages)

,id_member_anonymous,text_content_anonymous,quantidade
0,f233cf8b1d4ede06f32199a7e0081bf5,This community was blocked in Brazil following...,838
1,c743967449a387ad2c1c7e03b2c45b36,This community was blocked in Brazil following...,532
2,7696d5103cdb8ac352d748a1db1126b0,This community was blocked in Brazil following...,530
3,1665e22b0f564cd46d343f7677014821,Bem vindo(a) ao grupo Ipirá Notícias. \n\nComp...,357
4,a398999c55f8d6f0c65760522ae12e45,This community was blocked in Brazil following...,308
...,...,...,...
18327,88d15f878ab3b7965752ff0ad296c331,"Atenção: Boicote Geral\n\nRepasse, Copie e col...",2
18328,2ca5e50e644ef3b09f675b71a8339153,"Atenção: Boicote Geral\n\nRepasse, Copie e col...",2
18329,22da7e558d613074dc701f8a40bc955b,Qual é a legitimidade de um processo eleitoral...,2
18330,27f5833380581fab3453c1acd26427d5,Tô falando ... Aquilo é false flag. E eles tem...,2


  20. Mensagens  idênticas  compartilhadas  pelo  mesmo  usuário  em  grupos distintos (e suas quantidades)

In [46]:
identical_messages_groups = duckdb.sql("""
  SELECT 
    id_member_anonymous, 
    text_content_anonymous, 
    COUNT(DISTINCT id_group_anonymous) AS grupos_distintos
  FROM telegram_df
  WHERE text_content_anonymous IS NOT NULL AND text_content_anonymous <> '' AND id_member_anonymous IS NOT NULL
  GROUP BY id_member_anonymous, text_content_anonymous
  HAVING COUNT(DISTINCT id_group_anonymous) > 1
  ORDER BY grupos_distintos DESC
""").df()

display(identical_messages_groups)

,id_member_anonymous,text_content_anonymous,grupos_distintos
0,a7840e7844020149e197272748965862,ALERTA\nESSE GRUPO FOI CRIADO PELA ESQUERDA CO...,16
1,326d0a2f1bc5a1ede446fcf5dc31ff2e,💣💥((((( URGENTÍSSIMO )))))💥💣\nESSE GRUPO É UMA...,14
2,326d0a2f1bc5a1ede446fcf5dc31ff2e,💣💣💣💥💥🔰BOMBA BOMBA🔰💥💥💣💣💣\nVCS ESTÃO ACHANDO QUE...,12
3,29dda76df3384c28a07e8bad1ee9ceed,BOA NOITE A TODOS. O QUE EU VOU DIZER É SÉRIO ...,11
4,4e93f1d3c83fd6041314517363b14ed5,Pessoal não envie seus comprovantes de votação...,10
...,...,...,...
12937,27f5833380581fab3453c1acd26427d5,Roraima está virando paraíso...\nGovernador do...,2
12938,1c0774aae1e3ce3b6ced876b4e614900,👆👆🔴😡🤮🤮VEJA ISSO! DAQUI A POUCO OS DEMÔNIOS VÃO...,2
12939,d36c39da7c9705400e5056e033363b65,Screenshot (12 de out. de 2022 22:42:07),2
12940,f479c092ac0e7ead34fb08af36708ec6,"""CPX"" escrito no boné de LULA quando ele subiu...",2


  21.  Os 30 unigramas, bigramas e trigramas mais compartilhados

In [48]:
# Unigramas
unigrams = duckdb.sql("""
  SELECT word AS ngram, COUNT(*) AS count
  FROM (
    SELECT UNNEST(regexp_split_to_array(text_content_anonymous, '\\s+')) AS word
    FROM telegram_df
    WHERE text_content_anonymous IS NOT NULL AND text_content_anonymous <> ''
  )
  GROUP BY ngram
  ORDER BY count DESC
  LIMIT 30
""").df()

print("Top 30 Unigrams:")
display(unigrams)

# Bigramas
bigrams = duckdb.sql("""
  SELECT bigram AS ngram, COUNT(*) AS count
  FROM (
    SELECT CONCAT_WS(' ', word1, word2) AS bigram
    FROM (
      SELECT 
        LAG(word) OVER (PARTITION BY id_message ORDER BY position) AS word1,
        word AS word2
      FROM (
        SELECT id_message, UNNEST(regexp_split_to_array(text_content_anonymous, '\\s+')) AS word, ROW_NUMBER() OVER () AS position
        FROM telegram_df
        WHERE text_content_anonymous IS NOT NULL AND text_content_anonymous <> ''
      )
    )
    WHERE word1 IS NOT NULL
  )
  GROUP BY ngram
  ORDER BY count DESC
  LIMIT 30
""").df()

print("Top 30 Bigrams:")
display(bigrams)

# Trigramas
trigrams = duckdb.sql("""
  SELECT trigram AS ngram, COUNT(*) AS count
  FROM (
    SELECT CONCAT_WS(' ', word1, word2, word3) AS trigram
    FROM (
      SELECT 
        LAG(word, 2) OVER (PARTITION BY id_message ORDER BY position) AS word1,
        LAG(word, 1) OVER (PARTITION BY id_message ORDER BY position) AS word2,
        word AS word3
      FROM (
        SELECT id_message, UNNEST(regexp_split_to_array(text_content_anonymous, '\\s+')) AS word, ROW_NUMBER() OVER () AS position
        FROM telegram_df
        WHERE text_content_anonymous IS NOT NULL AND text_content_anonymous <> ''
      )
    )
    WHERE word1 IS NOT NULL AND word2 IS NOT NULL
  )
  GROUP BY ngram
  ORDER BY count DESC
  LIMIT 30
""").df()

print("Top 30 Trigrams:")
display(trigrams)

Top 30 Unigrams:


,ngram,count
0,de,325103
1,e,245433
2,a,244867
3,o,242071
4,que,228826
5,do,141206
6,para,114385
7,é,96876
8,não,96508
9,no,92723


Top 30 Bigrams:


,ngram,count
0,o que,18844
1,of the,18199
2,que o,18133
3,in Brazil,17437
4,a decision,17427
5,Superior Electoral,17426
6,the Superior,17421
7,Brazil following,17420
8,Court (TSE).,17420
9,following a,17420


Top 30 Trigrams:


,ngram,count
0,Superior Electoral Court,17420
1,the Superior Electoral,17419
2,Electoral Court (TSE).,17419
3,a decision of,17416
4,in Brazil following,17416
5,Brazil following a,17415
6,decision of the,17414
7,This community was,17414
8,following a decision,17414
9,community was blocked,17413


22. As 30 mensagens mais positivas (distintas)

In [53]:
top_30_positive_messages = duckdb.sql("""
  SELECT DISTINCT text_content_anonymous
  FROM telegram_df
  WHERE sentiment = 1
  ORDER BY sharings DESC
  LIMIT 30
""").df()

display(top_30_positive_messages)

,text_content_anonymous
0,This community was blocked in Brazil following...
1,"""E conhecereis a verdade, e a verdade vos libe..."
2,Bem vindo(a) ao grupo Ipirá Notícias. \n\nComp...
3,Vídeo de Maria Teresa D Valente
4,"Para quem gosta de MEDICINA NATURAL, esse é, S..."
5,Foto de Maria Teresa D Valente
6,O Meu amigo NÃO sou infiltrado ... meu voto fo...
7,INVISTA CONOSCO E RECEBA✅\n SEU LUCRO ESTÁ SEG...
8,"🚨Pessoal, estamos correndo contra o tempo para..."
9,"Desculpa, pessoal, eu sei que esse pode não se..."


23. As 30 mensagens mais negativas (distintas)

In [54]:
top_30_negative_messages = duckdb.sql("""
  SELECT DISTINCT text_content_anonymous
  FROM telegram_df
  WHERE sentiment = -1
  ORDER BY sharings DESC
  LIMIT 30
""").df()

display(top_30_negative_messages)

,text_content_anonymous
0,Grupo Para Organizar 200 MIL Eleitores do Bols...
1,BOA NOITE A TODOS. O QUE EU VOU DIZER É SÉRIO ...
2,Regras do Grupo \n\n1° Regra do grupo é respei...
3,Notícia de dentro do quartel. Os militares JÁ ...
4,Ola pessoal acessem o nosso grupo. \n✅Sem ofen...
5,🟢 PASSAPORTE SANITÁRIO 🟢\n\nRegistro doses da ...
6,*ATENÇÃO*\nFaça chegar a Brasília-DF \nVários ...
7,Regras do Grupo \n\n1° Regra do grupo é respei...
8,Cuidado pessoal tão postando em todos os grupo...
9,ATENÇÃO !!! ALERTA A TODOS!!\n\nA ESQUERDA EST...


24. O usuário mais otimista

In [61]:
most_optimistic_user = duckdb.sql("""
  SELECT id_member_anonymous, SUM(sentiment) AS total_sentiment
  FROM telegram_df
  WHERE id_member_anonymous IS NOT NULL
  GROUP BY id_member_anonymous
  ORDER BY total_sentiment DESC
  LIMIT 1
""").df()

print(most_optimistic_user)

                id_member_anonymous  total_sentiment
0  1665e22b0f564cd46d343f7677014821           3371.0


25. O usuário mais negativista

In [64]:
most_negative_user = duckdb.sql("""
  SELECT id_member_anonymous, SUM(sentiment) AS total_sentiment
  FROM telegram_df
  WHERE id_member_anonymous IS NOT NULL
  GROUP BY id_member_anonymous
  ORDER BY total_sentiment ASC
  LIMIT 1
""").df()

print(most_negative_user)

                id_member_anonymous  total_sentiment
0  c743967449a387ad2c1c7e03b2c45b36          -1677.0


26. As 30 maiores mensagens

In [66]:
top_30_distinct_largest_messages = duckdb.sql("""
  SELECT DISTINCT text_content_anonymous, caracteres
  FROM telegram_df
  ORDER BY caracteres DESC
  LIMIT 30
""").df()

display(top_30_distinct_largest_messages)


,text_content_anonymous,caracteres
0,PROPOSTAS DE BOLSONARO PARA 2023 - POSTADAS AG...,4096
1,Amados!\nOs olhos do mundo estavam voltados pa...,4093
2,OLE OLE OLE OLAAAA LULA LULAAAAAOLE OLE OLE OL...,4092
3,PETISTAS COMENDO MULHER D QUEM TÁ NOS BLOQUEIO...,4092
4,"ANTES DE 🇨🇳LULA🇨🇳, MIL VEZES 🇧🇷BOLSONARO🇧🇷, ma...",4090
5,O príncipe Dado Ruspoli também foi ator e no f...,4089
6,"Nada para efetuar seguros, porém, unicamente o...",4084
7,🚨ELEIÇÕES DE 2022 NO BRASIL 🇧🇷 DIVULGADO O ESC...,4080
8,Bolsonaro é um porco seboso e seus apoiadores ...,4078
9,Bolsonaro é um porco seboso e seus apoiadores ...,4077


27. As 30 menores mensagens

In [67]:
top_30_smallest_messages = duckdb.sql("""
  SELECT DISTINCT text_content_anonymous, caracteres
  FROM telegram_df
  WHERE text_content_anonymous IS NOT NULL AND text_content_anonymous <> ''
  ORDER BY caracteres ASC
  LIMIT 30
""").df()

display(top_30_smallest_messages)

,text_content_anonymous,caracteres
0,😂,1
1,👁,1
2,🤮,1
3,🧐,1
4,🙏,1
5,✅,1
6,😜,1
7,🟡,1
8,👆,1
9,🔜,1


28. O dia em que foi publicado a maior quantidade de mensagens

In [75]:
most_active_day = duckdb.sql("""
  SELECT CAST(date_message AS DATE) AS day, COUNT(*) AS message_count
  FROM telegram_df
  GROUP BY day
  ORDER BY message_count DESC
  LIMIT 1
""").df()

print(most_active_day)

         day  message_count
0 2022-10-02          25040


29. As mensagens que possuem as palavras “FACÇÃO” e “CRIMINOSA”

In [77]:
facção_criminosa_messages = duckdb.sql("""
  SELECT text_content_anonymous
  FROM telegram_df
  WHERE LOWER(text_content_anonymous) LIKE '%facção%'
    AND LOWER(text_content_anonymous) LIKE '%criminosa%'
""").df()

display(facção_criminosa_messages)

,text_content_anonymous
0,"Pensem!!!!!!'Se Houve fraude ,Nada foi feito ..."
1,#lulaladrãoseulugarénaprisão \n#PTFACÇÃOCRIMIN...
2,"Se depois de ler esse resumo do Plano do PT, v..."
3,"Se depois de ler esse resumo do Plano do PT, v..."
4,É exatamente isso que a facção criminosa CHAM...
...,...
124,https://youtu.be/aujth0dMkRA\n\nA treta das rá...
125,https://youtu.be/aujth0dMkRA\n\nA treta das rá...
126,https://youtu.be/gWNaArqZsmk\n\nChanceler da U...
127,Agressões covardes é termos uma facção crimino...


30. As mensagens que possuem a palavra “SEGURANÇA”

In [78]:
seguranca_messages = duckdb.sql("""
  SELECT text_content_anonymous
  FROM telegram_df
  WHERE LOWER(text_content_anonymous) LIKE '%segurança%'
""").df()

display(seguranca_messages)

,text_content_anonymous
0,⚡️🇧🇷 Actus Liberandi - Ordem De Cristo 🇧🇷\n\n✨...
1,⚡️🇧🇷 Actus Liberandi - Ordem De Cristo 🇧🇷\n\n✨...
2,No Telegram entrem em Configurações > Privacid...
3,No Telegram entrem em Configurações > Privacid...
4,🟡 Brasil no Radar\n\n▶️ O governo anunciou ont...
...,...
2470,🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨 \n\n *Urgente*\n\n...
2471,Profissão médica instada a dizer às mulheres a...
2472,"TSE, o relatório das Forças Armadas e o caso M..."
2473,"TSE, o relatório das Forças Armadas e o caso M..."
